| ![image.png](attachment:68c7e366-9d45-47ff-838c-0c1c38dc08cb.png) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;          |        <em><font size=5>Department of Modern Management and <br> Information Technology </font></em><br>  <font size=3>College of Arts, Media and Technology,<br> Chiang Mai University<br></font> Midterm Examination, Academic Year 2021 <br> Business Data Mining 954471
|:- |-: 
|<strong> ctober 18th, 2021 8:00 - 23:59 </strong>| <strong>(Total 35 Points) </strong>

<b>Name_________________Namphueng__Auawatcharo__________ Student ID_________622110186___________________________</b>

Instructions: 

-	This exam is worth 35% of your final grade.
-	This exam consists of 5 Questions
-	Finish this exam, download it as .ipynb file and send it to my FB messenger.
-	Write your student ID as filename. 
-	Allow anything.
-	<b>Please do this exam alone and be honest to yourself. </b>
-	The time allowed students to leave the testing room after the exam is open to copy it.
-	Students who cheat in any way will be prosecuted by the CMU regulation BE 2554, which governs student behavior and describes discipline during the exam period. The proctor must report any suspected cheating to the director.
<br>

### <em>Score Sheet:</em>
|<font size=3> Question|<font size=3> Full Mark|<font size=3> Student’s Mark|
|:- |:-:|:-:
<font size=3> Q4 |<font size=3> 10|
<font size=3> Total|<font size=3> 10|








In [19]:
import numpy as np
import pandas as pd

In [20]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# **Q4**
#### From this food images ("../input/food41/images"), please create an image classifier to classify 2 types of food choosing by yourself. (Show me the accuracy of each type of food) 
<b><font color=red>Both types of food must be more than 50% to get full score.</font></b>

# **Preprocessing**

In [21]:
!ls /kaggle/input/food41/images*

In [22]:
!mkdir train
!mkdir train/donuts
!mkdir train/steak

In [23]:
!mkdir test
!mkdir test/donuts
!mkdir test/steak

**TRAIN**

In [24]:
import glob 
donutsfile = np.array(glob.glob("/kaggle/input/food41/images/donuts/*.*"))
steakfile = np.array(glob.glob("/kaggle/input/food41/images/steak/*.*"))

In [25]:
donuts = np.array([int(item.split("/")[-1].split(".")[0]) for item in donutsfile])
steak = np.array([int(item.split("/")[-1].split(".")[0]) for item in steakfile])

In [26]:
import shutil
for i in range(len(donuts)-200):
    shutil.copy(donutsfile[i], '/kaggle/working/train/donuts/')
for i in range(len(steak)-200):
    shutil.copy(steakfile[i], '/kaggle/working/train/steak/')

**TEST**

In [27]:
for i in range(800,len(donuts)):
    shutil.copy(donutsfile[i],'/kaggle/working/test/donuts')
for i in range(800,len(steak)):
    shutil.copy(steakfile[i],'/kaggle/working/test/steak')

In [28]:
!ls /kaggle/working/train/donuts/* | wc -l

In [29]:
!ls /kaggle/working/train/steak/* | wc -l

In [30]:
!ls /kaggle/working/test/donuts/* | wc -l

In [31]:
!ls /kaggle/working/test/steak/* | wc -l

# **CNN**

In [32]:
#Images Augumentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        #rescale = 1.0/255.0,   # Intensity Normalized
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180) #หมุนภาพ
        zoom_range = 0.2, # Randomly zoom image #ซูมภาพ
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,   # randomly flip images
        validation_split=0.2) #แบ่่ง validation ไปใช้ใน deep learning 

In [33]:
#data generator
batch_size = 35
img_height = 100 
img_width  = 100 
train_dir = './train'

#train set
train_generator = data_generator.flow_from_directory(
    train_dir ,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', # label มีสองกลุ่มคือ dog กับ cat
    subset='training') # set as training data

#validation set
validation_generator = data_generator.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', # label มีสองกลุ่มคือ dog กับ cat
    subset='validation') # set as validation data

In [34]:
train_generator.class_indices

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
# preparing the layers in the Convolutional Deep Neural Network

def create_model():
    tf.random.set_seed(0)
    tf.compat.v1.reset_default_graph() # Clear Model
    #แบ่ง layers 
    model = Sequential()
    model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape)) #32 kernal ขนาด 3x3
    model.add(MaxPooling2D(pool_size = (2, 2))) 
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(rate = 0.2))
    
    model.add(Flatten()) 
    
    model.add(Dense(units = 16, activation = 'relu'))
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dense(units = 32, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [36]:
model = create_model()
fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 150)

In [43]:
import matplotlib.pyplot as plt
# plotting accuracy and validation accuracy
accuracy = fitted_model.history['accuracy']
val_accuracy = fitted_model.history['val_accuracy']
plt.plot(range(len(accuracy)), accuracy, 'b-', label = 'accuracy')
plt.plot(range(len(val_accuracy)), val_accuracy, 'r-', label = 'val_accuracy')
plt.legend()

In [44]:
import glob 
filedonuts = glob.glob("/kaggle/working/test/donuts/*.jpg")
filesteak = glob.glob("/kaggle/working/test/steak/*.jpg")

**Donuts Prediction**

In [45]:
from tensorflow.keras.preprocessing import image
# testing the model
p_donuts = []
for filename in filedonuts:
    test_image = image.load_img(filename, target_size = (img_height, img_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    p_donuts.append(np.round(model.predict(test_image)[0][0]))

In [46]:
#donuts accuracy
p_donuts = np.array(p_donuts)
sum(p_donuts==0)/len(p_donuts)

**Steak Prediction**

In [47]:
from tensorflow.keras.preprocessing import image
# testing the model
p_steak = []
for filename in filesteak:
    test_image = image.load_img(filename, target_size = (img_height, img_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    p_steak.append(np.round(model.predict(test_image)[0][0]))

In [48]:
#steak accuracy
p_steak = np.array(p_steak)
sum(p_steak==1)/len(p_steak)